## Oefeningen 6: Indices en efficiëntie

In [ ]:
# Hack the sys path for import
import sys
sys.path.insert(0,'..')

In [ ]:
import pandas as pd
from database import Database
db = Database("../oefeningen5en6.db")

In [ ]:
pd.options.display.max_colwidth = None

In [ ]:
db.query("DROP INDEX IF EXISTS IDX_Achternaam")

### Oefening 6.1: Executieplan zonder index

In [ ]:
# Bij detail staat SCAN Personen; de hele tabel wordt gescand...
db.query("""
EXPLAIN QUERY PLAN
SELECT * FROM Personen WHERE Achternaam = 'Jansen'
""")

### Oefening 6.2: Index op Achternaam

In [ ]:
# Maak index aan met CREATE INDEX
db.query("CREATE INDEX IDX_Achternaam ON Personen(Achternaam);")

In [ ]:
# Check of de index is aangemaakt
db.query("PRAGMA index_list(Personen);")

In [ ]:
# Merk op dat detail nu SEARCH ... USING INDEX vermeld.
db.query("""
EXPLAIN QUERY PLAN
SELECT * FROM Personen WHERE Achternaam = 'Jansen'
""")

### Oefening 6.3: Zoeken op Naam

In [ ]:
# Zoeken op voornaam resulteert nog steeds in een scan...
db.query("""
EXPLAIN QUERY PLAN
SELECT * FROM Personen WHERE Naam = 'Anna'
""")

### Oefening 6.4: Zoeken op Naam en Achternaam

In [ ]:
db.query("""
EXPLAIN QUERY PLAN
SELECT * FROM Personen WHERE Naam = 'Anna' AND Achternaam = 'Jansen'
""")

### Extra: Index op meerdere kolommen

In [ ]:
db.query("DROP INDEX IF EXISTS IDX_Achternaam")
db.query("DROP INDEX IF EXISTS IDX_VolledigeNaam")

In [ ]:
# Maak de multi-index aan
db.query("CREATE INDEX IDX_VolledigeNaam ON Personen(Naam, Achternaam)")

In [ ]:
# Check de velden van de index
db.query("PRAGMA index_info(IDX_VolledigeNaam);")

In [ ]:
# Index wordt gebruikt bij filters op Naam en Achternaam
db.query("""
EXPLAIN QUERY PLAN
SELECT * FROM Personen WHERE Naam = 'Anna' AND Achternaam = 'Jansen'
""")

In [ ]:
# Index wordt ook gebruikt bij filters alleen op Naam
db.query("""
EXPLAIN QUERY PLAN
SELECT * FROM Personen WHERE Naam = 'Anna'
""")

In [ ]:
# Maar... Index wordt NIET gebruikt bij filters alleen op Achternaam
db.query("""
EXPLAIN QUERY PLAN
SELECT * FROM Personen WHERE Achternaam = 'Jansen'
""")

In [ ]:
# Maar... Index wordt ook NIET gebruikt bij filters met OR
db.query("""
EXPLAIN QUERY PLAN
SELECT * FROM Personen WHERE Naam = 'Anna' OR Achternaam = 'Jansen'
""")